#Install the libraries

In [ ]:
# Install the libraries
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9

#Load the Data Set

In [ ]:
import os
from datasets import Dataset

def load_custom_dataset(data_dir):
    data = {"text": []}  # Initialize the dictionary with only "text" attribute

    file_list = os.listdir(data_dir)
    for file_name in file_list:
        if file_name == '.ipynb_checkpoints':
          continue
        print(file_name)
        file_path = os.path.join(data_dir, file_name)
        with open(file_path, "r") as file:
            lines = file.readlines()
            input_line = lines[0].strip()  # Get the input prompt
            output_lines = lines[1:]  # Get the structured slide content
            output_content = "".join(output_lines).strip()
            data["text"].append(input_line + "\n" + output_content)  # Combine input and output into a single string
    return Dataset.from_dict(data)

data_dir = "/content/DataSet"

# Load your custom dataset
custom_dataset = load_custom_dataset(data_dir)
print(custom_dataset)


decolonization.txt
data_mining.txt
data_cleaning.txt
neural_networks.txt
epistemology.txt
age_of_enlightenment.txt
ai_in_robotics.txt
deep_learning.txt
environmental_geography.txt
5g_technology.txt
cartography.txt
ancient_civilizations.txt
cuisine_and_gastronomy.txt
space_race.txt
data_science.txt
data_visualization.txt
existentialism.txt
music_and_society.txt
physical_geography.txt
computer_vision.txt
language_and_linguistics.txt
feature_engineering.txt
urban_planning.txt
cloud_computing.txt
art_history.txt
reinforcement_learning.txt
predictive_analytics.txt
mythology.txt
stoicism.txt
augmented_reality_(ar).txt
space_exploration.txt
aesthetics.txt
digital_revolution.txt
data_analysis.txt
climatology.txt
internet_of_things_(iot).txt
quantum_computing.txt
ethics.txt
electric_vehicles.txt
utilitarianism.txt
industrial_revolution.txt
cold_war.txt
natural_language_processing_(nlp).txt
artificial_intelligence.txt
folklore_and_traditions.txt
geomorphology.txt
blockchain_technology.txt
religi

#Import falcon model, tokenizer and peft

In [ ]:
import torch

"""
1) By using AutoModelForCausalLM, you can automatically load various pre-trained models for tasks like text generation, completion, and other sequence-to-sequence tasks.

2) AutoTokenizer: This class is also from the Hugging Face Transformers library and is used to automatically load a tokenizer for a specific pre-trained language model.
Tokenizers are essential for converting raw text into numerical representations that can be fed into the language model.
Different language models may require different tokenization techniques, so using AutoTokenizer allows you to automatically load the correct tokenizer based on the pre-trained model you want to use.


3) GenerationConfig: This class is a configuration class from the Hugging Face Transformers library used for specifying generation settings when generating text using a language model.
It allows you to control parameters like temperature, top-k sampling, top-p sampling, and beam search, which influence how the model generates the output text
"""
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig


#  This is the name of the pre-trained language model that we want to use
model_name = "ybelkada/falcon-7b-sharded-bf16"

# Bits and Bytes quantization is a quantization technique used in deep learning models to reduce memory and computational requirements

# Creating the Bits and Bytes configuration:



# This creates a configuration object for the Bits and Bytes quantization method with specific parameters:
bnb_config = BitsAndBytesConfig(

    # This tells the model to load the model weights in 4-bit format to reduce memory usage.
    load_in_4bit=True,

    # This sets the 4-bit quantization type to "nf4" (narrow-form 4-bit quantization).
    bnb_4bit_quant_type="nf4",

    # This specifies that 4-bit computations will be performed using 16-bit floating-point data type (float16).
    bnb_4bit_compute_dtype=torch.float16,
)

# Loading the pre-trained model:
model = AutoModelForCausalLM.from_pretrained(

    # This loads the pre-trained Falcon model specified by model_name
    model_name,

    # The quantization_config parameter is set to use the Bits and Bytes configuration created earlier.
    quantization_config=bnb_config,

    # The trust_remote_code=True option allows loading the model without inspecting the remote code
    trust_remote_code=True
)

'''
This disables the cache for the model configuration.
Caching can speed up inference but may consume more memory. In this case, it's set to False, indicating that the cache won't be used. '''
model.config.use_cache = False

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
'''
This line initializes the tokenizer by loading it from the Hugging Face model hub using the from_pretrained method.
The model_name variable contains the name or identifier of the pre-trained model, and trust_remote_code=True ensures that the remote code (associated with the model) can be trusted.
'''
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# In this line, we are setting the pad_token of the tokenizer to the value of eos_token. This is a common practice to ensure consistent handling of padding tokens throughout the model's input pipeline.
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
'''
peft is a library developed by Hugging Face that provides support for the LOcal Reparameterization with Autoregressive factorization (LORA) technique,
which aims to improve the training efficiency and stability of transformers for various tasks.
'''

from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(

    # This is a hyperparameter that controls the stability of the LORA method. A higher value of lora_alpha increases the stability of LORA, but it may also lead to increased training time and memory usage
    lora_alpha = 16,

    # The dropout rate to be applied during the LORA method. Dropout is a regularization technique used during training to prevent overfitting
    lora_dropout = 0.1,

    # The rank of the low-rank approximation for the self-attention matrix. A higher value of lora_r results in a higher-rank approximation, which can improve training efficiency.
    r = 64,

    # The type of bias to be used in the attention computation. The available options are "none" (no bias) or "learnable" (learnable bias).
    bias="none",

    # The type of task for which LORA is being used. In this case, it is set to "CAUSAL_LM", indicating that LORA is used for a causal language modeling task.
    task_type="CAUSAL_LM",

    # A list of transformer modules that will be targeted by the LORA method
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

#Set the training Arguments

In [ ]:
from transformers import TrainingArguments

# In this code snippet, we are creating a TrainingArguments object from the transformers library, which is used to store and manage various training-related configurations for fine-tuning a model.

training_arguments = TrainingArguments(

    # The directory where the trained model and training logs will be saved automaticaly
    output_dir = "./results",

    # The batch size to be used per GPU during training
    per_device_train_batch_size = 3,

    # The number of steps to accumulate gradients before performing an optimization step. This is used for gradient accumulation, which can be useful when dealing with limited GPU memory
    gradient_accumulation_steps = 3,

    # The name of the optimization algorithm to use during training. In this case, it is set to "paged_adamw_32bit"
    optim = "paged_adamw_32bit",

    # The number of steps between each checkpoint save during training
    save_steps = 10,

    # The number of steps between each logging update during training
    logging_steps = 10,

    # The learning rate to be used during optimization
    learning_rate=2e-4,

    # A boolean flag indicating whether to use mixed precision training with automatic mixed precision (AMP). This can speed up training by using lower-precision computations
    fp16=True,

    # The maximum norm for the gradients. Gradients exceeding this norm will be clipped to prevent exploding gradients
    max_grad_norm = 0.3,

    # The total number of training epochs
    num_train_epochs = 20,

    # The proportion of training steps to use for warmup in the learning rate scheduler
    warmup_ratio = 0.03,

    # A boolean flag indicating whether to group the input data by length to improve training efficiency.
    group_by_length=True,

    # The type of learning rate scheduler to use during training. In this case, it is set to "constant", which means the learning rate will remain constant throughout training.
    lr_scheduler_type = "constant",
)

In [ ]:
'''
In this code snippet, we are creating a SFTTrainer object from the trl (Tune and Regularize Language models) library.
SFTTrainer is a trainer class specifically designed for training language models using self-training with fine-tuning.
'''

from trl import SFTTrainer

trainer = SFTTrainer(

    # The language model that will be trained. In this case, it's the pre-trained model loaded using AutoModelForCausalLM with quantization config
    model = model,

    # The training dataset. It should be a dataset object containing the input data for training
    train_dataset = custom_dataset,

    # The configuration for the PEFT (Parameter Efficient Fine-Tuning) regularization method
    peft_config = peft_config,

    # he name of the attribute in the dataset object that contains the text data to be used for training. In this case, it is set to "text"
    dataset_text_field = "text",

    # The maxmimum sequence lenght of a combined input and output
    max_seq_length = 2000,

    # The tokenizer to be used for tokenizing the input data. It's the tokenizer created using AutoTokenizer
    tokenizer = tokenizer,

    # The training arguments, which is an object of the TrainingArguments class. It contains various hyperparameters and settings related to the training process
    args = training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:119: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/77 [00:00<?, ? examples/s]

#Begin fine tuning

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
# Start the training process
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


#Use the fine tuned model

In [ ]:

# Step 1: Set the model in evaluation mode
model.eval()

#
device = "cuda:0"

# Step 2: Preprocess the input data and convert it into input tensors
prompt = "###Human: Generate 9 slides on the topic 'Honda VS Toyota'. Each slide should have a slide number:, a heading: and content:"

inputs = tokenizer(prompt, return_tensors="pt").to(device)
#inputs = tokenizer(prompt, return_tensors="pt")

input_ids = inputs["input_ids"].to(model.device)
attention_mask = inputs["attention_mask"].to(model.device)

# Step 3: Generate text using the model
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=500,  # Set the maximum length of the generated text
        num_beams=1,    # Set the number of beams for beam search
        early_stopping=True
    )

# Step 4: Decode the generated output tensor back into text
generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)

print(generated_text)



#inputs = tokenizer(text, return_tensors="pt",return_token_type_ids=False).to(device)
#outputs = model.generate(**inputs, max_new_tokens=50)
#response = tokenizer.decode(outputs[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


###Human: Generate 9 slides on the topic 'Honda VS Toyota'. Each slide should have a slide number:, a heading: and content:. ###Assistant:
Slide 1: 
Heading: Introduction 
Content: Honda and Toyota are two of the most popular car brands in the world. They have been competing against each other for decades, with each brand trying to outdo the other in terms of quality, performance, and safety.

Slide 2: 
Heading: History 
Content: Honda and Toyota have been around since the 1950s, with Honda being founded in 1959 and Toyota being founded in 1937. Both brands have a long history of success, with Honda being the first brand to introduce the hybrid engine and Toyota being the first brand to introduce the electric vehicle.

Slide 3: 
Heading: Models 
Content: Honda and Toyota offer a wide range of models, from small cars to large SUVs. Honda offers 19 models, while Toyota offers 36 models.

Slide 4: 
Heading: Performance 
Content: Honda and Toyota both offer high-performance models, with Ho